This notebook serves as an example of how to load and manipulate the [QM9 dataset](https://figshare.com/collections/Quantum_chemistry_structures_and_properties_of_134_kilo_molecules/978904) using a `Dataset` object.

# Imports

In [ ]:
import os
import numpy as np

from ase import Atoms

# Initialize the database

In [ ]:
from colabfit.tools.database import MongoDatabase

client = MongoDatabase('colabfit_database', nprocs=6)

# Data loading

## Define the properties and reader functions

In [ ]:
client.insert_property_definition({
    'property-id': 'qm9-property',
    'property-title': 'A, B, C, mu, alpha, homo, lumo, gap, r2, zpve, U0, U, H, G, Cv',
    'property-description': 'Geometries minimal in energy, corresponding harmonic frequencies, dipole moments, polarizabilities, along with energies, enthalpies, and free energies of atomization',
    'a':     {'type': 'float', 'has-unit': True, 'extent': [], 'required': True, 'description': 'Rotational constant A'},
    'b':     {'type': 'float', 'has-unit': True, 'extent': [], 'required': True, 'description': 'Rotational constant B'},
    'c':     {'type': 'float', 'has-unit': True, 'extent': [], 'required': True, 'description': 'Rotational constant C'},
    'mu':    {'type': 'float', 'has-unit': True, 'extent': [], 'required': True, 'description': 'Dipole moment'},
    'alpha': {'type': 'float', 'has-unit': True, 'extent': [], 'required': True, 'description': 'Isotropic polarizability'},
    'homo':  {'type': 'float', 'has-unit': True, 'extent': [], 'required': True, 'description': 'Energy of Highest occupied molecular orbital (HOMO)'},
    'lumo':  {'type': 'float', 'has-unit': True, 'extent': [], 'required': True, 'description': 'Energy of Lowest occupied molecular orbital (LUMO)'},
    'gap':   {'type': 'float', 'has-unit': True, 'extent': [], 'required': True, 'description': 'Gap, difference between LUMO and HOMO'},
    'r2':    {'type': 'float', 'has-unit': True, 'extent': [], 'required': True, 'description': 'Electronic spatial extent'},
    'zpve':  {'type': 'float', 'has-unit': True, 'extent': [], 'required': True, 'description': 'Zero point vibrational energy'},
    'u0':    {'type': 'float', 'has-unit': True, 'extent': [], 'required': True, 'description': 'Internal energy at 0 K'},
    'u':     {'type': 'float', 'has-unit': True, 'extent': [], 'required': True, 'description': 'Internal energy at 298.15 K'},
    'h':     {'type': 'float', 'has-unit': True, 'extent': [], 'required': True, 'description': 'Enthalpy at 298.15 K'},
    'g':     {'type': 'float', 'has-unit': True, 'extent': [], 'required': True, 'description': 'Free energy at 298.15 K'},
    'cv':    {'type': 'float', 'has-unit': True, 'extent': [], 'required': True, 'description': 'Heat capacity at 298.15 K'},
    'smiles-relaxed':    {'type': 'string', 'has-unit': False, 'extent': [], 'required': True, 'description': 'SMILES for relaxed geometry'},
    'inchi-relaxed':     {'type': 'string', 'has-unit': False, 'extent': [], 'required': True, 'description': 'InChI for relaxed geometry'},
})

In [ ]:
client.get_property_definition('qm9-property')['definition']

## Defining a `property_map`

In [ ]:
property_map = {
    'qm9-property': {
        # Property Definition field: {'field': ASE field, 'units': ASE-readable units}
        'a':     {'field': 'A',     'units': 'GHz'},
        'b':     {'field': 'B',     'units': 'GHz'},
        'c':     {'field': 'C',     'units': 'GHz'},
        'mu':    {'field': 'mu',    'units': 'Debye'},
        'alpha': {'field': 'alpha', 'units': 'Bohr*Bohr*Bohr'},
        'homo':  {'field': 'homo',  'units': 'Hartree'},
        'lumo':  {'field': 'lumo',  'units': 'Hartree'},
        'gap':   {'field': 'gap',   'units': 'Hartree'},
        'r2':    {'field': 'r2',    'units': 'Bohr*Bohr'},
        'zpve':  {'field': 'zpve',  'units': 'Hartree'},
        'u0':    {'field': 'U0',    'units': 'Hartree'},
        'u':     {'field': 'U',     'units': 'Hartree'},
        'h':     {'field': 'H',     'units': 'Hartree'},
        'g':     {'field': 'G',     'units': 'Hartree'},
        'cv':    {'field': 'Cv',    'units': 'cal/mol/K'},
        'smiles-relaxed': {'field': 'SMILES_relaxed', 'units': None},
        'inchi-relaxed': {'field': 'SMILES_relaxed',  'units': None},
    }
}

In [ ]:
def reader(file_path):
    # A function for returning a list of ASE a
    
    properties_order = [
        'tag', 'index', 'A', 'B', 'C', 'mu', 'alpha', 'homo', 'lumo', 'gap', 'r2', 'zpve', 'U0', 'U', 'H', 'G', 'Cv'
    ]
        
#     images = []
    with open(file_path, 'r') as f:
        lines = [_.strip() for _ in f.readlines()]
        
        na = int(lines[0])
        properties = lines[1].split()
        
        symbols = []
        positions = []
        partial_charges = []
        
        for line in lines[2:2+na]:
            split = line.split()
            split = [_.replace('*^', 'e') for _ in split]  # Python-readable scientific notation
            
            # Line order: symbol, x, y, z, charge
            symbols.append(split[0])
            positions.append(split[1:4])
            partial_charges.append(split[-1])
            
        positions = np.array(positions)
        partial_charges = np.array(partial_charges, dtype=float)
                
        atoms = Atoms(symbols=symbols, positions=positions)
        
        atoms.info['mulliken_partial_charges'] = partial_charges
        
        name = os.path.splitext(os.path.split(file_path)[-1])[0]

        atoms.info['name'] = name
        
        for pname, val in zip(properties_order[2:], properties[2:]):
            atoms.info[pname] = float(val)
            
        frequencies = np.array(lines[-3].split(), dtype=float)
        atoms.info['frequencies'] = frequencies
                
        smiles = lines[-2].split()
        inchi  = lines[-1].split()
        
        atoms.info['SMILES']    = smiles[0]
        atoms.info['SMILES_relaxed'] = smiles[1]
        atoms.info['InChI']     = inchi[0]
        atoms.info['InChI_relaxed']  = inchi[1]
        
        yield atoms
#         images.append(atoms)
    
#     return images

## Load and insert the data

In [ ]:
from colabfit.tools.database import load_data

images = list(load_data(
    file_path='/home/jvita/scripts/colabfit/data/quantum-machine/qm9',
    file_format='folder',
    name_field='name',  # key in Configuration.info to use as the Configuration name
    elements=['H', 'C', 'N', 'O', 'F'],    # order matters for CFG files, but not others
    default_name='qm9',  # default name with `name_field` not found
    reader=reader,
    glob_string='*.xyz',
    verbose=True
))

In [ ]:
from colabfit.tools.property_settings import PropertySettings

pso = PropertySettings(
    method='DFT/B3LYP/6-31G(2df,p)',
    description='QM9 property settings calculation',
    files=None,
    labels=['DFT', 'B3LYP', '6-31G(2df,p)'],
)

ids = list(client.insert_data(
    images,
    property_map=property_map,
    property_settings={'qm9-property': pso},
    generator=False,
    verbose=True
))

# Defining `ConfigurationSet`s

In [ ]:
co_ids1 = client.get_data('configurations', fields='_id', query={'names': {'$regex': 'dsC7O2H10nsd'}}, ravel=True).tolist()
co_ids2 = client.get_data('configurations', fields='_id', query={'names': {'$regex': 'dsgdb9nsd'}}, ravel=True).tolist()

print('Configuration set 1', '(dsC7O2H10nsd):'.rjust(15), f'{len(co_ids1)}'.rjust(7))
print('Configuration set 2', '(dsgdb9nsd):'.rjust(15), f'{len(co_ids2)}'.rjust(7))

In [ ]:
cs_id1 = client.insert_configuration_set(
    co_ids1,
    description='Isomers of C7O2H10',
    verbose=True
)
cs_id1

In [ ]:
cs_id2 = client.insert_configuration_set(
    co_ids2,
    description='The subset of all 133,885 species with up to '\
    'nine heavy atoms (CONF) out of the GDB-17 chemical universe '\
    'of 166 billion organic molecules',
    verbose=True
)
cs_id2

In [ ]:
cs1 = client.get_configuration_set(cs_id1)['configuration_set']

print(cs1.description)

for k,v in cs1.aggregated_info.items():
    print(k, v)

In [ ]:
cs2 = client.get_configuration_set(cs_id2)['configuration_set']

print(cs2.description)

for k,v in cs2.aggregated_info.items():
    print(k, v)

# Defining `Dataset`s

In [ ]:
all_co_ids, all_pr_ids = list(zip(*ids))
len(all_pr_ids)

In [ ]:
ds_id = client.insert_dataset(
    cs_ids=[cs_id1, cs_id2],
    pr_ids=all_pr_ids,
    name='QM9',
    authors=['Raghunathan Ramakrishnan', 'Pavlo Dral', 'Matthias Rupp', 'O. Anatole von Lilienfeld'],
    links=[
        'https://www.nature.com/articles/sdata201422',
        'https://figshare.com/collections/Quantum_chemistry_structures_and_properties_of_134_kilo_molecules/978904'
    ],
    description="The QM9 dataset, split into the GDB-9 molecules and the C7O2H10 isomers",
    resync=True,
    verbose=True,
)
ds_id

In [ ]:
dataset = client.get_dataset(ds_id, resync=True, verbose=True)['dataset']

for k,v in dataset.aggregated_info.items():
    print(k,v)

# Exploration

In [ ]:
client.get_statistics(
    ['qm9-property.a', 'qm9-property.b', 'qm9-property.c'],
    ids=dataset.property_ids,
    verbose=True
)

In [ ]:
client.plot_histograms(
    ['qm9-property.a', 'qm9-property.b', 'qm9-property.c',],
    ids=dataset.property_ids
)

In [ ]:
clean_config_sets, clean_property_ids = client.filter_on_properties(
    ds_id=ds_id,
    filter_fxn=lambda x: (x['qm9-property']['a']['source-value'] < 20) and x['qm9-property']['b']['source-value'] < 10,
    fields=['qm9-property.a.source-value', 'qm9-property.b.source-value'],
    verbose=True
)

In [ ]:
new_cs_ids = []
for cs in clean_config_sets:
    new_cs_ids.append(client.insert_configuration_set(cs.configuration_ids, cs.description, verbose=True))
    
print(new_cs_ids)

In [ ]:
ds_id_clean = client.insert_dataset(
    cs_ids=new_cs_ids,
    pr_ids=clean_property_ids,
    name='QM9_filtered',
    authors=dataset.authors,
    links=[
        'https://www.nature.com/articles/sdata201422',
        'https://figshare.com/collections/Quantum_chemistry_structures_and_properties_of_134_kilo_molecules/978904'
    ],
    description="The QM9 dataset, filtered by removing values with a>=20 or b>=10",
    resync=True,
    verbose=True,
)
ds_id_clean

In [ ]:
clean_ds = client.get_dataset(ds_id_clean)['dataset']
len(clean_ds.property_ids)

In [ ]:
client.plot_histograms(
    ['qm9-property.a', 'qm9-property.b', 'qm9-property.c'],
    ids=clean_ds.property_ids,
    verbose=True
)